In [1]:
import importlib
from numpy.core.numeric import Inf
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from NNet.utils.writeNNet import writeNNet
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as patches
import time
import importlib
import models
import datasets
import math
import utils
from utils import *
import tasks

In [29]:
importlib.reload(datasets)
importlib.reload(tasks)

def training_all():
    task_classes = [
                    (tasks.LinnosTask, 301), 
                    ]
    for task_class, time_out in task_classes:
        print("task_class: ", task_class, "normal")
        task_class(time_out = time_out).train_and_verify() # normal
        print("task_class: ", task_class, "vanilla")
        task_class(add_counterexample = True, incremental_training = True, time_out=time_out).train_and_verify() # vanilla
        print("task_class: ", task_class, "spec data")
        task_class(add_counterexample = True, incremental_training = True, batch_counterexample = True, time_out=time_out).train_and_verify() # spec data
        print("task_class: ", task_class, "early reject")
        task_class(add_counterexample = True, incremental_training = True, batch_counterexample = True, early_rejection = True, time_out=time_out).train_and_verify() # w/ early rejection
training_all()

task_class:  <class 'tasks.LinnosTask'> normal
Using cuda device
Epoch 1
-------------------------------
Train Error: 
 Accuracy: 52.8%, Avg loss: 0.000520 

time: 1.130941390991211 301
Epoch 2
-------------------------------


In [18]:
def to_device(data, device):
    return tuple(tensor.to(device) for tensor in data) if isinstance(data, tuple) else data.to(device)

def test(model, data, batch_size, compute_loss, verbose=False, compute_error=False, device="cuda"):
    dataloader = DataLoader(data, batch_size=batch_size)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    digit_size, digit_correct = 0, 0
    positive = 0
    with torch.no_grad():
        for X, y in dataloader:
            X = to_device(X, device)
            y = to_device(y, device)
            if compute_error and compute_loss is not None:
                acc_cnt, loss = compute_loss(model, X, y)
                test_loss += loss.cpu().item() if torch.is_tensor(loss) else loss
                correct += acc_cnt.cpu().item() if torch.is_tensor(acc_cnt) else acc_cnt
    if not compute_error:
        return None, None
    test_loss /= num_batches
    acc = correct * 1. / size
    if verbose:
        print(f"Test Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return acc, test_loss

def test_model(model, training_data, time_out, task=None, device="cuda"):

    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    print('model size: {:.3f}KB'.format(param_size / 1024))
    
    def eval_loss(model, X, y):
        # normal_X, counter_X = X
        # normal_y, counter_y = y
        normal_X = X
        normal_y = y
        normal_acc_cnt, normal_loss = task.compute_normal_loss(model, normal_X, normal_y)
        return normal_acc_cnt, normal_loss

    model = model.to(device)
    model.eval()

    eval_fn = eval_loss if task is not None else None
    
    ### latency ###
    batch_size=1
    start = time.time()
    acc, loss = test(model, training_data, batch_size, eval_fn,
                     compute_error=False, device=device)
    end = time.time()
    print("latency time (us): {:.5f}".format((end - start)/len(training_data)*1e6))
    # print("accuracy: {:.9f}".format(acc))
    # print("l2 error: {:.9f}".format(loss))
    
    ### throughput ###
    start = time.time()
    rep_times = 100
    batch_size = len(training_data)
    for t in range(rep_times):
        acc, loss = test(model, training_data, batch_size, eval_fn, compute_error=False, device=device)
    end = time.time()
    print("throughput: {:.5f}".format(len(training_data)/((end - start)/rep_times)))

    acc, loss = test(model, training_data, batch_size, eval_fn, compute_error=True, device=device)
    print("accuracy: {:.9f}".format(acc))
    print("l2 error: {:.9f}".format(loss))


def accuracy_comparison(task_class, time_out, verified, device="cuda"):

    training_data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=True)
    print(training_data)
    task = task_class(add_counterexample = verified, incremental_training = verified, batch_counterexample = verified, time_out=time_out) # wo finetune
    task.load_model("../model/"+task.save_name+".pth")

    print(task.save_name)
    print("data size: ", len(training_data))

    test_model(task.model, training_data, time_out, task=task, device="cuda")

In [26]:
accuracy_comparison(tasks.LinnosTask, 301, False)

LinnosTask00000-1_301
data size:  11974
model size: 365.629KB
latency time (us): 76.55843
throughput: 180297.69084
accuracy: 0.955904460
l2 error: 0.000059425


In [28]:
accuracy_comparison(tasks.LinnosTask, 301, True)

LinnosTask11100-1_301
data size:  11974
model size: 365.629KB
latency time (us): 76.98284
throughput: 180033.40625
accuracy: 0.972273259
l2 error: 0.000052859


In [55]:
def train(model, data, batch_size, compute_loss, verbose=False, device = "cuda"):
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    for batch, (X, y) in enumerate(dataloader):
        X = to_device(X, device)
        y = to_device(y, device)
        acc_cnt, loss = compute_loss(model, X, y)
        # print(acc_cnt, "/", len(X))
        correct += acc_cnt.cpu().item() if torch.is_tensor(acc_cnt) else acc_cnt
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().item() if torch.is_tensor(loss) else loss
        # if verbose and batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    acc = correct * 1. / size
    if verbose:
        print(f"Train Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    return acc, train_loss

def baseline(threshold_perc=90, device = "cuda", epochs=100):
    model = models.FC(1, 9, 256, 2)
    loss_fn = nn.CrossEntropyLoss()
    def compute_loss(model, x, y):
        y = y.squeeze(dim=1)
        pred = model(x)
        # print(pred)
        # print(y)
        # print(pred.argmax(1))
        acc_cnt = (pred.argmax(1) == y).type(torch.float).sum().item()
        # print(acc_cnt)
        # assert False
        loss = loss_fn(pred, y)
        return acc_cnt, loss

    training_data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=False)
    print(np.shape(training_data.xs))
    print(np.max(training_data.ys))
    print(np.min(training_data.ys))
    print(np.mean(training_data.ys))
    print(np.median(training_data.ys))
    print(np.percentile(training_data.ys, threshold_perc))
    
    threshold = np.percentile(training_data.ys, threshold_perc)
    training_data.ys = training_data.ys > threshold
    training_data.ys = training_data.ys.astype(int)
    for ep in range(epochs):
        print("epoch: ", ep)
        acc, train_loss = train(model, training_data, 5000, compute_loss, verbose=True, device=device)
    return model

In [65]:
%time linnos_base = baseline(threshold_perc=98, device="cuda", epochs=1000)

(47895, 9)
1.0
0.0
0.00701667
0.00058624975
0.04543687999248505
epoch:  0
Train Error: 
 Accuracy: 40.7%, Avg loss: 0.709129 

epoch:  1
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.475997 

epoch:  2
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.312938 

epoch:  3
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.209358 

epoch:  4
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.149577 

epoch:  5
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.117561 

epoch:  6
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.102098 

epoch:  7
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.094054 

epoch:  8
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.087106 

epoch:  9
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.081595 

epoch:  10
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.078143 

epoch:  11
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.074670 

epoch:  12
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.072555 

epoch:  13
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.070589 

epoch:  14
Train Error: 
 Accuracy: 98.0%, Avg los

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056783 

epoch:  128
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057157 

epoch:  129
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056697 

epoch:  130
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056653 

epoch:  131
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056801 

epoch:  132
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057081 

epoch:  133
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056863 

epoch:  134
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057096 

epoch:  135
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056662 

epoch:  136
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057208 

epoch:  137
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057023 

epoch:  138
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056869 

epoch:  139
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056980 

epoch:  140
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.057159 

epoch:  141
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056933 

epoch:  142
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056239 

epoch:  255
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056375 

epoch:  256
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056226 

epoch:  257
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056156 

epoch:  258
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055916 

epoch:  259
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056516 

epoch:  260
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056280 

epoch:  261
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056461 

epoch:  262
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056121 

epoch:  263
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056524 

epoch:  264
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056330 

epoch:  265
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056281 

epoch:  266
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056187 

epoch:  267
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056341 

epoch:  268
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056403 

epoch:  269
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055870 

epoch:  382
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055682 

epoch:  383
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055979 

epoch:  384
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055476 

epoch:  385
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055646 

epoch:  386
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055571 

epoch:  387
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055802 

epoch:  388
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055821 

epoch:  389
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055381 

epoch:  390
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055902 

epoch:  391
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.056119 

epoch:  392
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055859 

epoch:  393
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055495 

epoch:  394
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055479 

epoch:  395
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055775 

epoch:  396
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055175 

epoch:  509
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055594 

epoch:  510
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055090 

epoch:  511
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055315 

epoch:  512
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055610 

epoch:  513
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054686 

epoch:  514
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055397 

epoch:  515
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055362 

epoch:  516
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055130 

epoch:  517
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055290 

epoch:  518
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055361 

epoch:  519
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055312 

epoch:  520
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055533 

epoch:  521
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055256 

epoch:  522
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055385 

epoch:  523
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054816 

epoch:  636
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055070 

epoch:  637
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054731 

epoch:  638
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054890 

epoch:  639
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054784 

epoch:  640
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054970 

epoch:  641
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055040 

epoch:  642
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054776 

epoch:  643
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055049 

epoch:  644
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054777 

epoch:  645
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054559 

epoch:  646
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054807 

epoch:  647
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054978 

epoch:  648
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054899 

epoch:  649
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.055450 

epoch:  650
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054801 

epoch:  763
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054460 

epoch:  764
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054872 

epoch:  765
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054747 

epoch:  766
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054669 

epoch:  767
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054921 

epoch:  768
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054556 

epoch:  769
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054685 

epoch:  770
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054832 

epoch:  771
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054589 

epoch:  772
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054690 

epoch:  773
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054760 

epoch:  774
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054642 

epoch:  775
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054933 

epoch:  776
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054399 

epoch:  777
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054377 

epoch:  890
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054634 

epoch:  891
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054135 

epoch:  892
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054278 

epoch:  893
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054689 

epoch:  894
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054350 

epoch:  895
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054176 

epoch:  896
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054330 

epoch:  897
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054160 

epoch:  898
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054525 

epoch:  899
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054413 

epoch:  900
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054255 

epoch:  901
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054285 

epoch:  902
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054642 

epoch:  903
Train Error: 
 Accuracy: 98.0%, Avg loss: 0.054825 

epoch:  904
Train Error: 
 Accuracy: 

In [64]:
%time linnos_base = baseline(threshold_perc=95, device="cuda", epochs=1000)

(47895, 9)
1.0
0.0
0.00701667
0.00058624975
0.04387019500136374
epoch:  0
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.520674 

epoch:  1
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.363172 

epoch:  2
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.268234 

epoch:  3
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.215354 

epoch:  4
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.189000 

epoch:  5
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.173617 

epoch:  6
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.160267 

epoch:  7
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.148280 

epoch:  8
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.139615 

epoch:  9
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.131280 

epoch:  10
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.125178 

epoch:  11
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.118839 

epoch:  12
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.113761 

epoch:  13
Train Error: 
 Accuracy: 95.0%, Avg loss: 0.108695 

epoch:  14
Train Error: 
 Accuracy: 95.0%, Avg los

Train Error: 
 Accuracy: 95.6%, Avg loss: 0.083189 

epoch:  128
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.083396 

epoch:  129
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.083421 

epoch:  130
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.083167 

epoch:  131
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.082996 

epoch:  132
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.082984 

epoch:  133
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082919 

epoch:  134
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.083515 

epoch:  135
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.083295 

epoch:  136
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.082501 

epoch:  137
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082711 

epoch:  138
Train Error: 
 Accuracy: 95.6%, Avg loss: 0.083299 

epoch:  139
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082971 

epoch:  140
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082867 

epoch:  141
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.083201 

epoch:  142
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082169 

epoch:  255
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082722 

epoch:  256
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082188 

epoch:  257
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082343 

epoch:  258
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082341 

epoch:  259
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082919 

epoch:  260
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082356 

epoch:  261
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082433 

epoch:  262
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082205 

epoch:  263
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082658 

epoch:  264
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082547 

epoch:  265
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.081822 

epoch:  266
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082595 

epoch:  267
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082369 

epoch:  268
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.082450 

epoch:  269
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082058 

epoch:  382
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081605 

epoch:  383
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.082017 

epoch:  384
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081944 

epoch:  385
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081666 

epoch:  386
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.081905 

epoch:  387
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081933 

epoch:  388
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081347 

epoch:  389
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.081545 

epoch:  390
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081856 

epoch:  391
Train Error: 
 Accuracy: 95.7%, Avg loss: 0.081841 

epoch:  392
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081886 

epoch:  393
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081665 

epoch:  394
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081542 

epoch:  395
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.081529 

epoch:  396
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 95.9%, Avg loss: 0.081059 

epoch:  509
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.080349 

epoch:  510
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080923 

epoch:  511
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080907 

epoch:  512
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080545 

epoch:  513
Train Error: 
 Accuracy: 95.8%, Avg loss: 0.080890 

epoch:  514
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.081122 

epoch:  515
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080746 

epoch:  516
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080781 

epoch:  517
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080815 

epoch:  518
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080844 

epoch:  519
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080844 

epoch:  520
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080825 

epoch:  521
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080817 

epoch:  522
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080688 

epoch:  523
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079757 

epoch:  636
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080281 

epoch:  637
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080177 

epoch:  638
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079879 

epoch:  639
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080287 

epoch:  640
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079602 

epoch:  641
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080152 

epoch:  642
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080051 

epoch:  643
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080072 

epoch:  644
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080402 

epoch:  645
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080552 

epoch:  646
Train Error: 
 Accuracy: 95.9%, Avg loss: 0.080035 

epoch:  647
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.080531 

epoch:  648
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079793 

epoch:  649
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079942 

epoch:  650
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079405 

epoch:  763
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079375 

epoch:  764
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079673 

epoch:  765
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079425 

epoch:  766
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079264 

epoch:  767
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079529 

epoch:  768
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079121 

epoch:  769
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079339 

epoch:  770
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079587 

epoch:  771
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079400 

epoch:  772
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079323 

epoch:  773
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079653 

epoch:  774
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079232 

epoch:  775
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079064 

epoch:  776
Train Error: 
 Accuracy: 96.0%, Avg loss: 0.079384 

epoch:  777
Train Error: 
 Accuracy: 

Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078718 

epoch:  890
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078611 

epoch:  891
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078910 

epoch:  892
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079071 

epoch:  893
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078887 

epoch:  894
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078595 

epoch:  895
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078683 

epoch:  896
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078546 

epoch:  897
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078912 

epoch:  898
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078824 

epoch:  899
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079004 

epoch:  900
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078836 

epoch:  901
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.079111 

epoch:  902
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078609 

epoch:  903
Train Error: 
 Accuracy: 96.1%, Avg loss: 0.078734 

epoch:  904
Train Error: 
 Accuracy: 

In [52]:
data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=False)
test_model(linnos_base, data, 300, device="cuda")

<class 'models.FC'>
model size: 12.008KB
latency time (us): 76.12961
throughput: 172489.91679
accuracy: 0.000000000
l2 error: 0.000000000


In [53]:
data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=False)
test_model(linnos_base, data, 300, device="cpu")

model size: 12.008KB
latency time (us): 35.37525
throughput: 172835.43905
accuracy: 0.000000000
l2 error: 0.000000000
